In [ ]:
#load the dataset
import csv

data = []
labels = []

file = open("IMDB Dataset.csv", 'r', encoding='utf8')
reader = csv.reader(file)

for line in reader:
    if line[1] == 'positive':
        data.append(line[0])
        labels.append('optimistic')
    elif line[1] == 'negative':
        data.append(line[0])
        labels.append('pessimistic')
    
print(len(data), len(labels))

In [ ]:
from bs4 import BeautifulSoup

cleanData = []

#remove html tags
for text in data:
    soup = BeautifulSoup(text, "html.parser")
    cleanData.append(soup.get_text())

#stopwords and punctuation should remain if using BERT
